In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.0/617.0 kB 6.7 MB/s eta 0:00:00


In [ ]:
import torch
import cv2
from ultralytics import YOLO
import os

# Set the CUDA device if needed
torch.cuda.set_device(0)

# Initialize the YOLO model
model = YOLO('yolov8n.pt')

# Open the input video file
video_capture = cv2.VideoCapture('/content/drive/MyDrive/vids/run.mp4')

# Define the output video file settings
output_file_path = '/content/drive/MyDrive/outputs/yolo_out_video.avi'
output_frames_per_second = 30.0
output_frame_size = (int(video_capture.get(3)), int(video_capture.get(4)))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file_path, fourcc, output_frames_per_second, output_frame_size)

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # Make predictions on the frame using the YOLO model
    results = model.predict(source=frame, conf=0.25)

    for box in results[0].boxes:
        class_id = results[0].names[box.cls.item()]
        if class_id=='person':
            coords = box.xyxy[0].tolist()
            prob = round(box.conf[0].item(), 2)

            # Draw bounding box and label on the frame
            x1, y1, x2, y2 = map(int, coords)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{class_id}: {prob}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    # Write the frame to the output video
    output_video.write(frame)

# Release the video capture and writer
video_capture.release()
output_video.release()
cv2.destroyAllWindows()


100%|██████████| 6.23M/6.23M [00:00<00:00, 153MB/s]

0: 640x384 1 person, 114.5ms
Speed: 12.7ms preprocess, 114.5ms inference, 346.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 7.5ms
Speed: 3.1ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 frisbee, 1 baseball bat, 8.3ms
Speed: 2.7ms preprocess, 8.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 frisbee, 8.0ms
Speed: 2.2ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 9.0ms
Speed: 1.9ms preprocess, 9.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 12.5ms
Speed: 2.0ms preprocess, 12.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 8.6ms
Speed: 2.2ms preprocess, 8.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 7.4ms
Speed

In [ ]:
import numpy as np
from collections import deque
import tensorflow as tf

from keras.models import load_model

In [ ]:
LRCN_model=load_model('/content/drive/MyDrive/Model/LRCN_Custom.h5')
current_model='LRCN'
LRCN_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_48 (TimeD  (None, 20, 64, 64, 16)    448       
 istributed)                                                     
                                                                 
 time_distributed_49 (TimeD  (None, 20, 16, 16, 16)    0         
 istributed)                                                     
                                                                 
 time_distributed_50 (TimeD  (None, 20, 16, 16, 16)    0         
 istributed)                                                     
                                                                 
 time_distributed_51 (TimeD  (None, 20, 16, 16, 32)    4640      
 istributed)                                                     
                                                                 
 time_distributed_52 (TimeD  (None, 20, 4, 4, 32)     

In [ ]:
IMAGE_HEIGHT,IMAGE_WIDTH=64,64 #original 64,64
SEQUENCE_LENGTH=20 #original 20
CLASS_LIST=['run','walk','crawl','jump']

In [ ]:
def predict_on_video(video_file_path,SEQUENCE_LENGTH):
  video_reader=cv2.VideoCapture(video_file_path)

  original_width=int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
  origial_height=int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frames_queue=deque(maxlen=SEQUENCE_LENGTH)
  predicted_class_name=''

  while video_reader.isOpened():
    ok,frame=video_reader.read()

    if not ok:
      break

    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255

    frames_queue.append(normalized_frame)

    if len(frames_queue)==SEQUENCE_LENGTH:
        predicted_label_prob=model.predict(np.expand_dims(frames_queue,axis=0))[0]

        predicted_label=np.argmax(predicted_label_prob)

        predicted_class_name=CLASS_LIST[predicted_label]


        if max(predicted_label_prob)<=0.8:
          print(str(max(predicted_label_prob))+" Class: "+predicted_class_name+" was deleted!")
          predicted_class_name=''
        else:
          print(str(max(predicted_label_prob))+" Class: "+predicted_class_name+" was kept!")
          return(predicted_class_name)
  video_reader.release()
  return(predicted_class_name)



In [ ]:
def vid_preprocess(video_file_path,startz):
  # Input video file
  input_video_path = video_file_path

  # Output video file
  output_video_path = 'temp.mp4'

  # Start and end durations in seconds
  start_duration = startz
  end_duration = start_duration+1

  # Open the input video file for reading
  cap = cv2.VideoCapture(input_video_path)

  # Get the frames per second (fps) and total frame count of the input video
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  # Calculate the start and end frames based on durations
  start_frame = int(start_duration * fps)
  end_frame = int(end_duration * fps)

  # Get the width and height of the video frames
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  # Define the codec and create a VideoWriter object to save the output video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs like 'XVID' or 'H264'
  out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

  # Loop through the frames and write the selected portion to the output video
  frame_count = 0
  while True:
      ret, frame = cap.read()
      if not ret:
          break

      frame_count += 1

      # Check if the current frame is within the specified duration
      if start_frame <= frame_count <= end_frame:
          out.write(frame)

      # Break the loop when we reach the end frame
      if frame_count == end_frame:
          break

  # Release the video objects
  cap.release()
  out.release()

  print("Video cut and saved successfully.")
  return(predict_on_video(output_video_path,SEQUENCE_LENGTH))


In [ ]:
# Set the CUDA device if needed
torch.cuda.set_device(0)

# Initialize the YOLO model
yolo_model = YOLO('yolov8n.pt')

# Open the input video file
vid_file='/content/drive/MyDrive/vids/testo.mp4'
video_capture = cv2.VideoCapture(vid_file)
frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))
clip_duration = frame_count / frame_rate


# Define the output video file settings
output_file_path = '/content/drive/MyDrive/outputs/human_action_detection.avi'
output_frames_per_second = 30.0
output_frame_size = (int(video_capture.get(3)), int(video_capture.get(4)))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file_path, fourcc, output_frames_per_second, output_frame_size)

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # Make predictions on the frame using the YOLO model
    results = yolo_model.predict(source=frame, conf=0.25)

    for box in results[0].boxes:
        class_id = results[0].names[box.cls.item()]
        if class_id=='person':
            coords = box.xyxy[0].tolist()
            prob = round(box.conf[0].item(), 2)
            x1, y1, x2, y2 = map(int, coords)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            current_time_ms = int(video_capture.get(cv2.CAP_PROP_POS_MSEC))
            current_time_secs = current_time_ms / 1000.0

            start_sec = current_time_secs
            end_sec = start_sec + 1
            if end_sec<=clip_duration:
                action_label=vid_preprocess(vid_file,start_sec)
            else:
              action_label="NA"
            # Draw bounding box and label on the frame
            x1, y1, x2, y2 = map(int, coords)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{class_id}: {prob}:{action_label}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    # Write the frame to the output video
    output_video.write(frame)

# Release the video capture and writer
video_capture.release()
output_video.release()
cv2.destroyAllWindows()


0: 640x640 (no detections), 9.1ms
Speed: 7.9ms preprocess, 9.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.1ms
Speed: 3.0ms preprocess, 10.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.5ms
Speed: 3.4ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.7ms
Speed: 3.0ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.1ms
Speed: 3.1ms preprocess, 12.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.5ms
Speed: 7.1ms preprocess, 9.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.2ms
Speed: 3.1ms preprocess, 10.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.1ms
Speed: 3.3ms preprocess, 10.1ms infer

Video cut and saved successfully.
1/1 [==============================] - 0s 21ms/step
0.9980792 Class: walk was kept!
1/1 [==============================] - 0s 23ms/step
0.9980866 Class: walk was kept!
1/1 [==============================] - 0s 27ms/step
0.99808216 Class: walk was kept!
1/1 [==============================] - 0s 37ms/step
0.9980823 Class: walk was kept!
1/1 [==============================] - 0s 29ms/step
0.9980825 Class: walk was kept!
1/1 [==============================] - 0s 30ms/step
0.99808025 Class: walk was kept!
1/1 [==============================] - 0s 29ms/step
0.99808013 Class: walk was kept!
Video cut and saved successfully.
1/1 [==============================] - 0s 28ms/step
0.9980792 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.9980866 Class: walk was kept!
1/1 [==============================] - 0s 29ms/step
0.99808216 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.9980823 Class: walk was kept!
1/1 

0.99808013 Class: walk was kept!


0: 640x640 2 persons, 15.1ms
Speed: 7.0ms preprocess, 15.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


Video cut and saved successfully.
1/1 [==============================] - 0s 36ms/step
0.99808717 Class: walk was kept!
1/1 [==============================] - 0s 27ms/step
0.99808204 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.99808204 Class: walk was kept!
1/1 [==============================] - 0s 29ms/step
0.99808216 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.99808025 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.99808013 Class: walk was kept!
1/1 [==============================] - 0s 31ms/step
0.9980768 Class: walk was kept!
Video cut and saved successfully.
1/1 [==============================] - 0s 32ms/step
0.99808717 Class: walk was kept!
1/1 [==============================] - 0s 31ms/step
0.99808204 Class: walk was kept!
1/1 [==============================] - 0s 29ms/step
0.99808204 Class: walk was kept!
1/1 [==============================] - 0s 28ms/step
0.99808216 Class: walk was kept

...

0: 640x640 1 person, 12.6ms
Speed: 3.4ms preprocess, 12.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


Video cut and saved successfully.
1/1 [==============================] - 0s 22ms/step
0.9887917 Class: crawl was kept!
1/1 [==============================] - 0s 24ms/step
0.98876697 Class: crawl was kept!
1/1 [==============================] - 0s 31ms/step
0.9888248 Class: crawl was kept!
1/1 [==============================] - 0s 32ms/step
0.98872584 Class: crawl was kept!
1/1 [==============================] - 0s 33ms/step
0.9878313 Class: crawl was kept!
1/1 [==============================] - 0s 31ms/step
0.9871724 Class: crawl was kept!
1/1 [==============================] - 0s 34ms/step
0.9867279 Class: crawl was kept!



0: 640x640 1 person, 10.6ms
Speed: 3.2ms preprocess, 10.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


Video cut and saved successfully.
1/1 [==============================] - 0s 23ms/step
0.9888671 Class: crawl was kept!
1/1 [==============================] - 0s 22ms/step
0.98881954 Class: crawl was kept!
1/1 [==============================] - 0s 29ms/step
0.9886778 Class: crawl was kept!
1/1 [==============================] - 0s 38ms/step
0.98787284 Class: crawl was kept!
1/1 [==============================] - 0s 24ms/step
0.98685503 Class: crawl was kept!
1/1 [==============================] - 0s 42ms/step
0.9867219 Class: crawl was kept!
1/1 [==============================] - 0s 38ms/step
0.98601973 Class: crawl was kept!



0: 640x640 1 cat, 10.5ms
Speed: 3.3ms preprocess, 10.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 cat, 10.2ms
Speed: 2.9ms preprocess, 10.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 cat, 9.9ms
Speed: 2.9ms preprocess, 9.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 cat, 9.1ms
Speed: 2.9ms preprocess, 9.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 cat, 8.9ms
Speed: 2.8ms preprocess, 8.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 cat, 9.0ms
Speed: 4.8ms preprocess, 9.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 cat, 10.5ms
Speed: 2.8ms preprocess, 10.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 cat, 9.4ms
Speed: 2.3ms preprocess, 9.4ms inference, 2.0ms postprocess per image at